In [1]:
%%HTML
<link rel="stylesheet" type="text/css" href="custom.css">
<style>
 .rise-enabled {background-color: #ffc;}
 .rise-enabled .text_cell {font-size: 120%;}
</style>
<script>
 MathJax.Hub.Config({displayAlign: 'left'});
</script>

In [2]:
# inicjalizacja
import ase
import ase.io
import numpy
from ase.visualize import view
from ase.build import bulk

#### Warsztaty modelowania w nanofizyce
----
## Drgania struktury krystalicznej - fonony

**Jan Łażewski**

Zakład Komputerowych Badań Materiałów,
Instytut Fizyki Jądrowej PAN, Kraków

<br />
<img src="FIGs/tyt.png" style="height:180px; margin-top:100px" />

<img src="FIGs/tyt0.png" style="height:650px" />

<img src="FIGs/tyt0.gif" style="height:650px" />

<img src="FIGs/tyt1.gif" style="height:650px" />

<img src="FIGs/tyt2.gif" style="height:650px" />

<img src="FIGs/tyt3.gif" style="height:650px" />

In [8]:
# przygotowanie rysunku
t=ase.io.read('FIGs/MnAs_softM48_10.traj',index=':')
v=view(t,viewer='nglview')

v.custom_colors({'Mn':'green','As':'blue'})
v.view._remote_call("setSize", target="Widget", args=["500px", "500px"])
#v.view.center_view()
v.view.background='#ffc'
v.view.parameters=dict(clipDist=-200)

# wyświetlenie rysunku
v

### Opis drgań kryształu w przybliżeniu harmonicznym - metoda bezpośrednia

<br />
K. Parlinski, Z.Q. Li, and Y. Kawazoe, Phys. Rev. Lett. **78**, 4063 (1997).

K. Parlinski, PHONON software (Cracow, Poland, 1997-2017), <br />
http://wolf.ifj.edu.pl/phonon/, http://www.computingformaterials.com/

### Rozwinięcie energii potencjalnej w przybliżeniu harmonicznym

$$
\newcommand{\mb}{\boldsymbol}
\newcommand{\h}{\hspace{-0.05cm}}
$$
W przybliżeniu harmonicznym energię potencjalną sieci krystalicznej
$E\bigl(\bigl\{ {\mb R}({\mb n},\mu)\bigr\}\bigr)$ można rozwinąć w szereg względem wychyleń ${\mb U}({\mb n},\mu)$ atomów <br />
z ich położeń równowagi:

$$
\begin{eqnarray}
E\bigl(\{ {\mb R}\}\bigr)\; =\; E_0\;\; + & & \h 
\sum_{{\mb n},\mu}\frac{\partial 
			 E\bigl(\bigl\{{\mb R}({\mb n},\mu)\bigr\}\bigr) }
			  {\partial {\mb U}({\mb n},\mu)}\,
		      {\mb U}({\mb n},\mu) \nonumber  \\
		         + & & \h
\frac{1}{2}\sum_{\substack{{\mb n},\mu \\ {\mb m},\nu}}
\frac{\partial^2E\bigl(\bigl\{ {\mb R}({\mb n},\mu)\bigr\}\bigr) }
    {\partial {\mb U}({\mb n},\mu)\,\partial {\mb U}({\mb m},\nu)}
\,{\mb U}({\mb n},\mu)\,{\mb U}({\mb m},\nu)\, \text{,}
%\label{eq:harmE}
\end{eqnarray}
$$

gdzie ${\mb R}({\mb n},\mu)$ oznacza pozycję atomu $\mu$ w komórce
prymitywnej ${\mb n}$, a sumowanie przebiega po całym krysztale.

### Macierz stałych siłowych

<br />
W rozwinięciu $\,E\bigl(\bigl\{ {\mb R}({\mb n},\mu)\bigr\}\bigr)$ pierwsze pochodne znikają z warunków minimum energii w równowadze, zaś drugie pochodne definiują macierz stałych siłowych:

$$
\begin{eqnarray}
    \Phi_{ij}({\mb n},\mu;{\mb m},\nu ) = 
    \frac{\partial^2 E\bigl(\bigl\{ {\mb R}({\mb n},\mu)\bigr\}\bigr) }
         {\partial U_i({\mb n},\mu)\,\partial U_j({\mb m},\nu)}\, \text{,} 
\end{eqnarray}
$$

reprezentującą oddziaływanie między atomami w krysztale. 


<br />
<img src="FIGs/springModel1.png" style="height:300px; margin-top:-45px; margin-left:650px" />

### Siły Hellmanna-Feynmana (HF)

<br />
Każde wychylenie ${\mb U}({\mb n},\mu)$, nawet pojedynczego atomu, powoduje powstanie sił HF na wszystkich atomach w układzie:

\begin{eqnarray}
 {\mb F}({\mb n},\mu) = - \frac{\partial E}{\partial {\mb R}({\mb n},\mu)}\, .
\end{eqnarray}

In [4]:
import ase.io
from ase.data import covalent_radii 
from ase.data.colors import cpk_colors
from ipywidgets import IntSlider, FloatSlider, interactive, fixed, HBox
from glob import glob


#scene.caption= """Right button drag or Ctrl-drag to rotate "camera" to view scene.
#To zoom, drag with middle button or Alt/Option depressed, or use scroll wheel.
#  On a two-button mouse, middle is left + right.
#Touch screen: pinch/extend to zoom, swipe or two-finger rotate."""

#clr={'Mn':color.green, 'As':color.blue}
clr={25:(0,0.51,0), 33:(0,0,0.95)}


def broom():
    for o in scene.objects:
        o.visible=False
        
class crystal:
    def __init__( self, trj, fscale=1, rscale=1 ):
        self.trj=trj
        self.rscale=rscale
        self.fscale=10**fscale
        self.n=0
        scene.fov=0.05
        scene.background=vec(1,1,0.8)
        scene.center=vec(*tuple(trj[0].get_center_of_mass()))
        self.atoms=[sphere(pos=vec(*tuple(r)), 
                           radius=self.rscale*covalent_radii[Z], 
                           color=vec(*clr[Z])) 
                     for Z, r in zip(trj[0].get_atomic_numbers(), trj[0].get_positions())]
        
        self.forces=[arrow(pos=vec(*tuple(r)), 
                           axis=vec(*tuple(f)), 
                           color=color.red) 
                     for r, f in zip(self.trj[0].get_positions(), self.fscale*trj[0].get_forces())]
   
    def update_frame( self, n=None, fscale=None, rscale=None ):
        if n is not None:
            self.n=n
        if fscale is not None:
            self.fscale=10**fscale
        if rscale is not None:
            self.rscale=rscale    
        for a, fv, r, f, Z in zip(self.atoms, self.forces, 
                               self.trj[self.n].get_positions(), 
                               self.fscale*self.trj[self.n].get_forces(),
                               self.trj[self.n].get_atomic_numbers()):
            a.pos=vec(*tuple(r))
            a.radius=self.rscale*covalent_radii[Z]
            fv.pos=vec(*tuple(r))
            fv.axis=vec(*tuple(f))
       
    def show(self, maxr=5, maxf=2.5):
        w1=interactive(self.update_frame, 
                       n=IntSlider(min=0,max=len(self.trj)-1,step=1,value=0,
                                   description='n:'), 
                       fscale=fixed(None), 
                       rscale=fixed(None));
        w2=interactive(self.update_frame, 
                       n=fixed(None), 
                       fscale=FloatSlider(min=1,max=maxf,value=1.0,
                                          description='siła:'), 
                       rscale=fixed(None));
        w3=interactive(self.update_frame, n=fixed(None), 
                       fscale=fixed(None), 
                       rscale=FloatSlider(min=0.1,max=maxr,value=1.0,
                                          description='promień:'));
        return HBox([w1, w2, w3])

In [6]:
from vpython import scene, vec, sphere, arrow, color

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
broom()
            
#trj=ase.io.Trajectory('data/md_T_1000.traj')
#trj=ase.io.read('FIGs/OUTCAR1', index=':')
trj=[ase.io.read(fn) for fn in sorted(glob('FIGs/OUTCAR_??'))]

c=crystal(trj)
c.show(1,2.5)

### Zależność sił HF od stałych siłowych

<br />
W przybliżeniu harmonicznym z rozwinięcia energii otrzymujemy zależność:

$$
\begin{eqnarray}
 F_i({\mb n},\mu) = -\sum_{{\mb m},\nu,j}
			\Phi_{ij}({\mb n},\mu;{\mb m},\nu )
			\,U_j({\mb m},\nu) .
\end{eqnarray}
$$


### Macierz dynamiczna

<br />
Macierz dynamiczną definiujemy jako transformatę Fouriera macierzy stałych
siłowych:

$$
\begin{eqnarray}
 {\mb D}({\mb k};\mu,\nu) = \frac{1}{\sqrt{M_\mu M_\nu}}
 	\sum_{\mb m} {\mb \Phi}(0,\mu;{\mb m},\nu)\;
	\text{e}^{-2\pi i{\mb k}\cdot[ {\mb R}(0,\mu)-{\mb R}({\mb m},\nu)]} \text{,}
\end{eqnarray}
$$

gdzie ${\mb k}$ jest wektorem falowym, $M_\mu$ i $M_\nu$ masami atomów
$\mu$ i $\nu$, zaś sumowanie przebiega po wszystkich atomach kryształu.

### Równanie własne

<br />
Rozwiązanie równania własnego:

$$
\begin{eqnarray}
 {\mb D}({\mb k})\,{\mb e}({\mb k},j) = \omega^2({\mb k},j)\,{\mb e}({\mb k},j) 
\end{eqnarray}
$$

poprzez diagonalizację macierzy dynamicznej daje częstości drgań 
$\,\omega({\mb k},j)$ 
<br />
i wektory polaryzacji $\,{\mb e}({\mb k},j)$.

<img src="FIGs/DC2.png" style="width:1600px" />

### Wyliczane własności dynamiczne 

<br />
Metodą bezpośrednią można wyliczyć między innymi następujące własności materiałowe:

* fononowe relacje dyspersji
* intensywności pików fononowych
* parametry Grüneissena
* wektory polaryzacji + animacja
* całkowite i cząstkowe widma DOS
* funkcje termodynamiczne: E, S, F, G, c$_\text{v}$
* czynniki Debye'a-Waller'a
* rozszerzalność cieplna
* rozpraszanie neutronów i X
* diagramy fazowe, granice stabilności struktur


# [PRZYKŁAD: przejście fazowe w MnAs](http://wolf.ifj.edu.pl/~lazewski/ws/warsztatyJL17.pdf)
    

In [9]:
!jupyter nbconvert FononyDFT.ipynb --to slides

[NbConvertApp] Converting notebook FononyDFT.ipynb to slides
[NbConvertApp] Writing 269884 bytes to FononyDFT.slides.html
